# Run this code in Google Colab




In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq


In [ ]:
import cirq
import sympy
import numpy as np

# Define numbers to partition
numbers = np.random.randint(-10, 100, 6)
n_qubits = len(numbers)

# Create qubits
qubits = [cirq.NamedQubit(f'q{i}') for i in range(n_qubits)]

# Build Cost Hamiltonian: H_C = (Σ a_i Z_i)² = Σ_i a_i² I + Σ_i,j a_i a_j Z_i Z_j (i < j)
cost_terms = []

# Add diagonal terms (i = j): Σ_i a_i² I
constant_term = sum(a*a for a in numbers)  # This adds to the total energy but doesn't affect evolution

# Add ZZ terms (i ≠ j): Σ_i,j a_i a_j Z_i Z_j
for i in range(n_qubits):
    for j in range(i+1, n_qubits):
        weight = 2 * numbers[i] * numbers[j]  # Factor of 2 because we're only counting i < j
        cost_terms.append(cirq.ZZ(qubits[i], qubits[j]) ** weight)

# Mixing Hamiltonian: H_B = Σ X_i
mixer = [cirq.X(q) for q in qubits]

# QAOA circuit construction
circuit = cirq.Circuit()

# Initial superposition
circuit.append(cirq.H.on_each(qubits))

# Variational parameters
gamma = sympy.Symbol('γ')
beta = sympy.Symbol('β')

# Add QAOA layers
for term in cost_terms:
    circuit.append(term ** gamma)
for term in mixer:
    circuit.append(term ** beta)

# Measurement
circuit.append(cirq.measure(*qubits, key='result'))




In [ ]:
cost_terms

[(cirq.ZZ**1456.0).on(cirq.NamedQubit('q0'), cirq.NamedQubit('q1')),
 (cirq.ZZ**1768.0).on(cirq.NamedQubit('q0'), cirq.NamedQubit('q2')),
 (cirq.ZZ**3588.0).on(cirq.NamedQubit('q0'), cirq.NamedQubit('q3')),
 (cirq.ZZ**4212.0).on(cirq.NamedQubit('q0'), cirq.NamedQubit('q4')),
 (cirq.ZZ**1144.0).on(cirq.NamedQubit('q0'), cirq.NamedQubit('q5')),
 (cirq.ZZ**1904.0).on(cirq.NamedQubit('q1'), cirq.NamedQubit('q2')),
 (cirq.ZZ**3864.0).on(cirq.NamedQubit('q1'), cirq.NamedQubit('q3')),
 (cirq.ZZ**4536.0).on(cirq.NamedQubit('q1'), cirq.NamedQubit('q4')),
 (cirq.ZZ**1232.0).on(cirq.NamedQubit('q1'), cirq.NamedQubit('q5')),
 (cirq.ZZ**4692.0).on(cirq.NamedQubit('q2'), cirq.NamedQubit('q3')),
 (cirq.ZZ**5508.0).on(cirq.NamedQubit('q2'), cirq.NamedQubit('q4')),
 (cirq.ZZ**1496.0).on(cirq.NamedQubit('q2'), cirq.NamedQubit('q5')),
 (cirq.ZZ**11178.0).on(cirq.NamedQubit('q3'), cirq.NamedQubit('q4')),
 (cirq.ZZ**3036.0).on(cirq.NamedQubit('q3'), cirq.NamedQubit('q5')),
 (cirq.ZZ**3564.0).on(cirq.NamedQ

In [ ]:
print(circuit)

                                           ┌──────────────────────────┐   ┌──────────────────────────┐   ┌───────────────────────────────────────┐   ┌──────────────────────────┐   ┌───────────────────────────┐
q0: ───H───ZZ──────────────ZZ───────────────ZZ─────────────────────────────ZZ─────────────────────────────ZZ──────────────────────────────────────────X^(β)─────────────────────────────────────────────────────────────────────────────────────────────────M('result')───
           │               │                │                              │                              │                                                                                                                                                 │
q1: ───H───ZZ^(1456.0*γ)───┼────────────────┼────────────ZZ────────────────┼────────────ZZ────────────────┼────────────ZZ─────────────────────────────ZZ─────────────────────────────X^(β)──────────────────────────────────────────────────────────────────M─────────────
  

In [ ]:
# Example optimization setup
simulator = cirq.Simulator()
params = [gamma, beta]

def energy_from_measurements(measurements):
    # Calculate (Σ a_i z_i)^2 from bitstrings
    return np.mean([(sum(numbers[i]*(-1)**b for i, b in enumerate(bs)))**2
                    for bs in measurements])

def objective(params):
    resolved = cirq.ParamResolver({'γ': params[0], 'β': params[1]})
    result = simulator.run(circuit, resolved, repetitions=500)
    return energy_from_measurements(result.measurements['result'])


In [ ]:
from scipy.optimize import minimize

initial = np.random.uniform(0, np.pi/2, 2)
result = minimize(objective, initial, method='COBYLA')
print("Optimal parameters:", result.x)


Optimal parameters: [ 2.19935745 -0.17078835]


In [ ]:
best_params = result.x
final_result = simulator.run(
    circuit,
    cirq.ParamResolver({'γ': best_params[0], 'β': best_params[1]}),
    repetitions=1000
)


In [ ]:
import numpy as np
import collections

# Assume `final_result` is obtained from running the circuit with optimized parameters:
# final_result = simulator.run(circuit, cirq.ParamResolver({'γ': best_params[0], 'β': best_params[1]}), repetitions=1000)
# And `numbers` is the list defined earlier, e.g., numbers = [3, 1, 4, 2, 2]
measurements = final_result.measurements['result']
n_qubits = len(numbers)

def bitstring_to_partition(bits):
    """
    Convention: 0 -> +1 (Subset A), 1 -> -1 (Subset B)
    """
    return [1 if bit == 0 else -1 for bit in bits]

def compute_cost(spin_assignment):
    """
    Computes the cost as the square of the imbalance.
    Imbalance S is given by sum(a_i * spin_i).
    """
    S = sum(a * spin for a, spin in zip(numbers, spin_assignment))
    return S * S

# Flatten the array to a list of bitstrings (each is a 1D array)
bitstring_list = [measurement for measurement in measurements]

# Dictionary to count frequency of each unique partition outcome and store its cost.
partition_costs = {}
for bits in bitstring_list:
    # Convert the bitstring into a tuple for dictionary keys.
    partition = tuple(bitstring_to_partition(bits))
    cost = compute_cost(partition)
    if partition in partition_costs:
        partition_costs[partition]['count'] += 1
    else:
        partition_costs[partition] = {'cost': cost, 'count': 1}

# Determine the partition(s) with the minimal cost.
min_cost = min(info['cost'] for info in partition_costs.values())
optimal_partitions = [p for p, info in partition_costs.items() if info['cost'] == min_cost]

try:
    from tabulate import tabulate
except ImportError:
    !pip install tabulate
    from tabulate import tabulate

# print("Minimum imbalance (cost):", min_cost)
# print("Optimal partition(s) and corresponding frequencies:")
# print(numbers)
# for partition in optimal_partitions:
#     sub_a = [numbers[i] for i in range(n_qubits) if partition[i] == 1]
#     sub_b = [numbers[i] for i in range(n_qubits) if partition[i] == -1]
#     print("Partition:", partition,
#           "First Partition", sub_a,
#           "Second Partition", sub_b,
#           "Sum Difference", sum(sub_a) - sum(sub_b),
#           "Frequency:", partition_costs[partition]['count'])

table_data = []
for partition in optimal_partitions:
    sub_a = [numbers[i] for i in range(n_qubits) if partition[i] == 1]
    sub_b = [numbers[i] for i in range(n_qubits) if partition[i] == -1]
    table_data.append([partition, sub_a, sub_b, sum(sub_a) - sum(sub_b), partition_costs[partition]['count']])

headers = ["Partition", "First Partition", "Second Partition", "Sum Difference", "Frequency"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))


+-----------------------+-------------------+--------------------+------------------+-------------+
| Partition             | First Partition   | Second Partition   |   Sum Difference |   Frequency |
+=======================+===================+====================+==================+=============+
| (-1, 1, 1, 1, -1, -1) | [28, 34, 69]      | [26, 81, 22]       |                2 |          29 |
+-----------------------+-------------------+--------------------+------------------+-------------+
| (1, -1, 1, 1, -1, -1) | [26, 34, 69]      | [28, 81, 22]       |               -2 |           7 |
+-----------------------+-------------------+--------------------+------------------+-------------+
| (-1, 1, -1, -1, 1, 1) | [28, 81, 22]      | [26, 34, 69]       |                2 |           3 |
+-----------------------+-------------------+--------------------+------------------+-------------+
| (1, -1, -1, -1, 1, 1) | [26, 81, 22]      | [28, 34, 69]       |               -2 |          19 |


In [ ]:
final_result.measurements

{'result': array([[1, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 1, 0],
        ...,
        [0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1]], dtype=int8)}

In [ ]:
try:
    import qiskit as qk
except ImportError:
    print("installing qiskit and related")
    !pip install --quiet qiskit qiskit-algorithms qiskit-aer
    print("installed qiskit.")


installing qiskit and related
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.0 MB/s eta 0:00:00
installed qiskit.


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Sampler
import numpy as np

class NumberPartitionQAOA:
    def __init__(self, numbers):
        self.numbers = numbers
        self.n_qubits = len(numbers)
        self.qubit_op = self._create_cost_operator()

    def _create_cost_operator(self):
        """Creates the cost Hamiltonian for number partitioning"""
        pauli_list = []

        # Create ZZ terms for all pairs of qubits
        for i in range(self.n_qubits):
            for j in range(i+1, self.n_qubits):
                # Coefficient is 2*a_i*a_j for i≠j terms
                coeff = 2.0 * self.numbers[i] * self.numbers[j]

                # Create Pauli string with Z's at positions i and j
                pauli_str = ['I'] * self.n_qubits
                pauli_str[i] = 'Z'
                pauli_str[j] = 'Z'
                pauli_list.append((coeff, ''.join(pauli_str)))

        # Convert to SparsePauliOp
        op_list = [(coeff, Pauli(pauli)) for coeff, pauli in pauli_list]
        return SparsePauliOp.from_list(op_list)

    def solve(self, reps=1, optimizer_maxiter=100):
        """Solve using QAOA"""
        # Initialize the optimizer
        optimizer = COBYLA(maxiter=optimizer_maxiter)

        # Create QAOA instance
        qaoa = QAOA(
            sampler=Sampler(),
            optimizer=optimizer,
            reps=reps
        )

        # Run QAOA
        result = qaoa.compute_minimum_eigenvalue(operator=self.qubit_op)

        # Get the result bitstring
        x = self._sample_most_likely(result.eigenstate)

        # Convert to partition
        partition = self._get_partition(x)

        return {
            'partition': partition,
            'cost': result.optimal_value,
            'bitstring': x,
            'success': result.success
        }

    def _sample_most_likely(self, state_vector):
        """Returns the most likely bitstring"""
        n = len(state_vector)
        k = int(np.log2(n))
        max_amplitude = max(np.abs(state_vector))
        max_i = np.where(np.abs(state_vector) == max_amplitude)[0][0]
        return bin(max_i)[2:].zfill(k)

    def _get_partition(self, bitstring):
        """Convert bitstring to two partitions"""
        partition_a = []
        partition_b = []
        for i, bit in enumerate(bitstring):
            if bit == '0':
                partition_a.append(self.numbers[i])
            else:
                partition_b.append(self.numbers[i])
        return partition_a, partition_b

# Example usage
if __name__ == "__main__":
    # Example numbers to partition
    numbers = [3, 1, 4, 2, 2]

    # Create and solve
    np_qaoa = NumberPartitionQAOA(numbers)
    result = np_qaoa.solve(reps=2)

    # Print results
    print("Numbers to partition:", numbers)
    print("\nPartition found:")
    print("  Subset A:", result['partition'][0])
    print("  Subset B:", result['partition'][1])
    print("  Sum A:", sum(result['partition'][0]))
    print("  Sum B:", sum(result['partition'][1]))
    print("  Difference:", abs(sum(result['partition'][0]) - sum(result['partition'][1])))
    print("\nOptimal cost:", result['cost'])
    print("Success:", result['success'])


TypeError: object of type 'float' has no len()